# Replication of Tables 1A and 1B - Smets & Wouters (2007)

This notebook replicates **Tables 1A and 1B** from:

> Smets, F. & Wouters, R. (2007). "Shocks and Frictions in US Business Cycles: A Bayesian DSGE Approach"
> *American Economic Review*, 97(3), 586-606.

- **Table 1A** (p.593): Prior and Posterior Distribution of Structural Parameters
- **Table 1B** (p.594): Prior and Posterior Distribution of Shock Processes

**Estimation Period**: 1966Q1 - 2004Q4

**Note on Posterior Statistics**:
- **Mode**: Available with `mh_replic=0` (current setting, fast)
- **Mean & Intervals**: Require MCMC sampling with `mh_replic>0` (slower, ~250,000 draws)

## 1. Setup and Configuration

In [1]:
# Imports
import numpy as np
import pandas as pd
import os
import sys
from pathlib import Path

# Add parent directory to path
sys.path.append(str(Path.cwd().parent.parent))

from direct_replication import DynareInterface

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [2]:
# Configure paths - MODIFY FOR YOUR INSTALLATION
import os
os.environ['OCTAVE_EXECUTABLE'] = r'C:\Program Files\GNU Octave\Octave-10.3.0\mingw64\bin\octave-cli.exe'

DYNARE_PATH = r'C:\dynare\6.5\matlab'
REPO_PATH = Path.cwd().parent.parent / 'repo'
MODEL_PATH = Path.cwd().parent / 'model'

print(f"Octave executable: {os.environ['OCTAVE_EXECUTABLE']}")
print(f"Dynare path: {DYNARE_PATH}")
print(f"Model path: {MODEL_PATH}")
print(f"Model exists: {MODEL_PATH.exists()}")

Octave executable: C:\Program Files\GNU Octave\Octave-10.3.0\mingw64\bin\octave-cli.exe
Dynare path: C:\dynare\6.5\matlab
Model path: c:\Users\HP\OneDrive\Escritorio\David Guzzi\Github\MECTMT11\direct_replication\model
Model exists: True


## 2. Reference Values from Paper

These are the exact values from Tables 1A and 1B of Smets & Wouters (2007).

**Note on Priors**: Prior distributions are inputs specified by the researcher, not estimated outputs.

In [3]:
# =============================================================================
# TABLE 1A - STRUCTURAL PARAMETERS
# Prior and Posterior Distribution of Structural Parameters
# =============================================================================

TABLE_1A_REFERENCE = {
    # Parameter: (Dynare name, Prior Distr, Prior Mean, Prior SD, Post Mode, Post Mean, Post 5%, Post 95%)
    'phi':       ('csadjcost', 'Normal',  4.00, 1.50, 5.48, 5.74, 3.97, 7.42),   # Investment adjustment cost
    'sigma_c':   ('csigma',    'Normal',  1.50, 0.37, 1.39, 1.38, 1.16, 1.59),   # Risk aversion
    'h':         ('chabb',     'Beta',    0.70, 0.10, 0.71, 0.71, 0.64, 0.78),   # Habit formation
    'xi_w':      ('cprobw',    'Beta',    0.50, 0.10, 0.73, 0.70, 0.60, 0.81),   # Wage Calvo probability
    'sigma_l':   ('csigl',     'Normal',  2.00, 0.75, 1.92, 1.83, 0.91, 2.78),   # Labor supply elasticity
    'xi_p':      ('cprobp',    'Beta',    0.50, 0.10, 0.65, 0.66, 0.56, 0.74),   # Price Calvo probability
    'iota_w':    ('cindw',     'Beta',    0.50, 0.15, 0.59, 0.58, 0.38, 0.78),   # Wage indexation
    'iota_p':    ('cindp',     'Beta',    0.50, 0.15, 0.22, 0.24, 0.10, 0.38),   # Price indexation
    'psi':       ('czcap',     'Beta',    0.50, 0.15, 0.54, 0.54, 0.36, 0.72),   # Capacity utilization
    'Phi':       ('cfc',       'Normal',  1.25, 0.12, 1.61, 1.60, 1.48, 1.73),   # Fixed cost
    'r_pi':      ('crpi',      'Normal',  1.50, 0.25, 2.03, 2.04, 1.74, 2.33),   # Taylor rule: inflation
    'rho':       ('crr',       'Beta',    0.75, 0.10, 0.81, 0.81, 0.77, 0.85),   # Taylor rule: smoothing
    'r_y':       ('cry',       'Normal',  0.12, 0.05, 0.08, 0.08, 0.05, 0.12),   # Taylor rule: output gap
    'r_Delta_y': ('crdy',      'Normal',  0.12, 0.05, 0.22, 0.22, 0.18, 0.27),   # Taylor rule: output growth
    'pi_bar':    ('constepinf','Gamma',   0.62, 0.10, 0.81, 0.78, 0.61, 0.96),   # SS quarterly inflation
    'beta_const':('constebeta','Gamma',   0.25, 0.10, 0.16, 0.16, 0.07, 0.26),   # 100(beta^-1 - 1)
    'l_bar':     ('constelab', 'Normal',  0.00, 2.00, -0.10, 0.53, -1.30, 2.32), # SS hours
    'gamma_bar': ('ctrend',    'Normal',  0.40, 0.10, 0.43, 0.43, 0.40, 0.45),   # Trend growth
    'alpha':     ('calfa',     'Normal',  0.30, 0.05, 0.19, 0.19, 0.16, 0.21),   # Capital share
}

print(f"Table 1A: {len(TABLE_1A_REFERENCE)} structural parameters defined")

Table 1A: 19 structural parameters defined


In [4]:
# =============================================================================
# TABLE 1B - SHOCK PROCESSES
# Prior and Posterior Distribution of Shock Processes
# =============================================================================

# Mapping from paper notation to Dynare shock names for std devs
# These are extracted from oo_.posterior_mode.shocks_std
SHOCK_STD_MAPPING = {
    'sigma_a': 'ea',      # Technology shock
    'sigma_b': 'eb',      # Preference shock  
    'sigma_g': 'eg',      # Government spending shock
    'sigma_I': 'eqs',     # Investment shock
    'sigma_r': 'em',      # Monetary policy shock
    'sigma_p': 'epinf',   # Price markup shock
    'sigma_w': 'ew',      # Wage markup shock
}

TABLE_1B_REFERENCE = {
    # Shock standard deviations
    # Parameter: (Dynare param name, Prior Distr, Prior Mean, Prior SD, Post Mode, Post Mean, Post 5%, Post 95%)
    'sigma_a':   ('ea',        'InvGamma', 0.10, 2.00, 0.45, 0.45, 0.41, 0.50),  # Technology shock
    'sigma_b':   ('eb',        'InvGamma', 0.10, 2.00, 0.24, 0.23, 0.19, 0.27),  # Preference shock
    'sigma_g':   ('eg',        'InvGamma', 0.10, 2.00, 0.52, 0.53, 0.48, 0.58),  # Government spending shock
    'sigma_I':   ('eqs',       'InvGamma', 0.10, 2.00, 0.45, 0.45, 0.37, 0.53),  # Investment shock
    'sigma_r':   ('em',        'InvGamma', 0.10, 2.00, 0.24, 0.24, 0.22, 0.27),  # Monetary policy shock
    'sigma_p':   ('epinf',     'InvGamma', 0.10, 2.00, 0.14, 0.14, 0.11, 0.16),  # Price markup shock
    'sigma_w':   ('ew',        'InvGamma', 0.10, 2.00, 0.24, 0.24, 0.20, 0.28),  # Wage markup shock
    
    # AR(1) persistence coefficients (these are regular parameters)
    'rho_a':     ('crhoa',     'Beta',     0.50, 0.20, 0.95, 0.95, 0.94, 0.97),  # Technology
    'rho_b':     ('crhob',     'Beta',     0.50, 0.20, 0.18, 0.22, 0.07, 0.36),  # Preference
    'rho_g':     ('crhog',     'Beta',     0.50, 0.20, 0.97, 0.97, 0.96, 0.99),  # Government spending
    'rho_I':     ('crhoqs',    'Beta',     0.50, 0.20, 0.71, 0.71, 0.61, 0.80),  # Investment
    'rho_r':     ('crhoms',    'Beta',     0.50, 0.20, 0.12, 0.15, 0.04, 0.24),  # Monetary policy
    'rho_p':     ('crhopinf',  'Beta',     0.50, 0.20, 0.90, 0.89, 0.80, 0.96),  # Price markup
    'rho_w':     ('crhow',     'Beta',     0.50, 0.20, 0.97, 0.96, 0.94, 0.99),  # Wage markup
    
    # MA coefficients
    'mu_p':      ('cmap',      'Beta',     0.50, 0.20, 0.74, 0.69, 0.54, 0.85),  # Price markup MA
    'mu_w':      ('cmaw',      'Beta',     0.50, 0.20, 0.88, 0.84, 0.75, 0.93),  # Wage markup MA
    
    # Cross-effect
    'rho_ga':    ('cgy',       'Beta',     0.50, 0.20, 0.52, 0.52, 0.37, 0.66),  # Govt spending on tech
}

print(f"Table 1B: {len(TABLE_1B_REFERENCE)} shock process parameters defined")

Table 1B: 17 shock process parameters defined


## 3. Execute Dynare and Extract Parameters

In [5]:
# Initialize Dynare interface
di = DynareInterface(DYNARE_PATH, str(MODEL_PATH))
print("Dynare interface initialized")

    _pyeval at line 57 column 10

    _pyeval at line 57 column 10

    _pyeval at line 57 column 10

Dynare interface initialized


In [6]:
# Run model
print("Running usmodel.mod...")
print("(This may take a few moments)\n")

di.run_model('usmodel.mod')

print("\nDynare execution completed")

Running usmodel.mod...
(This may take a few moments)


Step 1: Closing Octave session to release file locks...
Waiting for Windows to release file handles...

Step 2: Cleaning up directories...
Searching for directories to clean up...
No directories found to clean up.

Step 3: Starting fresh Octave session...
    _pyeval at line 57 column 10

    _pyeval at line 57 column 10

    _pyeval at line 57 column 10

✓ Octave session ready

Step 4: Running Dynare estimation...
Command: dynare usmodel nograph
(This may take several minutes...)

Starting Dynare (version 6.5).
Calling Dynare with arguments: nograph
Starting preprocessing of the model file ...
Found 40 equation(s).
Evaluating expressions...
Computing static model derivatives (order 1).
Normalizing the static model...
Finding the optimal block decomposition of the static model...
11 block(s) found:
  9 recursive block(s) and 2 simultaneous block(s).
  the largest simultaneous block has 13 equation(s)
                               

In [7]:
# Extract estimated parameters (posterior mode)
params_df = di.get_parameters()

# Convert to dictionary for easier access
estimated_params = dict(zip(params_df['parameter'], params_df['value']))

print(f"Extracted {len(estimated_params)} parameters from Dynare")
print("\nKey parameters:")
for param in ['csigma', 'chabb', 'cprobw', 'cprobp', 'crpi', 'crr', 'calfa']:
    if param in estimated_params:
        print(f"  {param}: {estimated_params[param]:.4f}")

Extracted 57 parameters from Dynare

Key parameters:
  csigma: 1.3952
  chabb: 0.7124
  cprobw: 0.7375
  cprobp: 0.6563
  crpi: 2.0295
  crr: 0.8153
  calfa: 0.1928


In [8]:
# =============================================================================
# EXTRACT SHOCK STANDARD DEVIATIONS
# These are stored separately in Dynare's estimation results
# =============================================================================

def extract_shock_std_devs(di):
    """
    Extract shock standard deviations from Dynare estimation results.
    
    These are stored in:
    - oo_.posterior_mode.shocks_std (after mode-finding)
    - Or from the diagonal of Sigma_e (shock covariance matrix)
    
    Returns:
        Dictionary mapping shock names to their estimated std devs
    """
    shock_stds = {}
    
    # Get shock names
    n_shocks = int(di.oc.eval('M_.exo_nbr', nout=1))
    shock_names = []
    for i in range(n_shocks):
        name = di.oc.eval(f'deblank(M_.exo_names{{{i+1}}})', nout=1)
        shock_names.append(str(name).strip())
    
    # Strategy 1: Try oo_.posterior_mode.shocks_std
    try:
        has_shocks_std = di.oc.eval(
            'isfield(oo_, "posterior_mode") && isfield(oo_.posterior_mode, "shocks_std")', 
            nout=1
        )
        if has_shocks_std:
            shocks_std = di.oc.eval('oo_.posterior_mode.shocks_std', nout=1)
            if hasattr(shocks_std, 'flatten'):
                shocks_std = shocks_std.flatten()
            for i, name in enumerate(shock_names):
                if i < len(shocks_std):
                    shock_stds[name] = float(shocks_std[i])
            print("Extracted shock std devs from oo_.posterior_mode.shocks_std")
            return shock_stds
    except Exception as e:
        print(f"Strategy 1 failed: {e}")
    
    # Strategy 2: Try sqrt of diagonal of Sigma_e
    try:
        has_sigma = di.oc.eval('isfield(M_, "Sigma_e")', nout=1)
        if has_sigma:
            sigma_e = di.oc.eval('M_.Sigma_e', nout=1)
            for i, name in enumerate(shock_names):
                shock_stds[name] = float(np.sqrt(sigma_e[i, i]))
            print("Extracted shock std devs from M_.Sigma_e diagonal")
            return shock_stds
    except Exception as e:
        print(f"Strategy 2 failed: {e}")
    
    # Strategy 3: Try estim_params_ structure
    try:
        # Check for estimated shock variances in estim_params_
        n_estim = int(di.oc.eval('size(estim_params_.var_exo, 1)', nout=1))
        if n_estim > 0:
            for i in range(n_estim):
                idx = int(di.oc.eval(f'estim_params_.var_exo({i+1}, 1)', nout=1))
                if idx <= len(shock_names):
                    shock_name = shock_names[idx - 1]
                    # Get the estimated value from bayestopt_
                    # The mode is stored in xparam1 after estimation
                    try:
                        mode_val = di.oc.eval(f'oo_.posterior_mode.parameters({i+1})', nout=1)
                        shock_stds[shock_name] = float(mode_val)
                    except:
                        pass
            if shock_stds:
                print("Extracted shock std devs from estim_params_")
                return shock_stds
    except Exception as e:
        print(f"Strategy 3 failed: {e}")
    
    print("Could not extract shock std devs - will use reference values")
    return shock_stds

# Extract shock standard deviations
shock_std_estimates = extract_shock_std_devs(di)

if shock_std_estimates:
    print("\nEstimated shock standard deviations:")
    for name, val in shock_std_estimates.items():
        print(f"  {name}: {val:.4f}")
else:
    print("\nNo shock std devs extracted - check Dynare output structure")

Strategy 1 failed: 'Struct' object is not callable
Extracted shock std devs from M_.Sigma_e diagonal

Estimated shock standard deviations:
  ea: 0.4518
  eb: 0.2425
  eg: 0.5200
  eqs: 0.4501
  em: 0.2398
  epinf: 0.1411
  ew: 0.2444


In [9]:
# =============================================================================
# CHECK FOR MCMC RESULTS (for posterior mean and intervals)
# =============================================================================

def check_mcmc_results(di):
    """
    Check if MCMC results are available (posterior mean and intervals).
    These require running with mh_replic > 0.
    
    Returns:
        Tuple of (has_mcmc, mcmc_results_dict)
    """
    mcmc_results = {
        'available': False,
        'param_means': {},
        'param_intervals': {},
        'shock_means': {},
        'shock_intervals': {}
    }
    
    try:
        # Check if MCMC was run
        has_mcmc = di.oc.eval(
            'isfield(oo_, "posterior_mean") && isfield(oo_.posterior_mean, "parameters")',
            nout=1
        )
        
        if has_mcmc:
            mcmc_results['available'] = True
            print("MCMC results available - extracting posterior means and intervals")
            
            # Extract parameter means
            n_params = int(di.oc.eval('length(oo_.posterior_mean.parameters)', nout=1))
            param_names_est = []
            for i in range(n_params):
                name = di.oc.eval(f'deblank(M_.param_names{{estim_params_.param_vals({i+1},1)}})', nout=1)
                param_names_est.append(str(name).strip())
            
            means = di.oc.eval('oo_.posterior_mean.parameters', nout=1).flatten()
            for i, name in enumerate(param_names_est):
                if i < len(means):
                    mcmc_results['param_means'][name] = float(means[i])
            
            # Extract HPD intervals if available
            try:
                has_hpd = di.oc.eval('isfield(oo_, "HPDinf") && isfield(oo_.HPDinf, "parameters")', nout=1)
                if has_hpd:
                    hpd_inf = di.oc.eval('oo_.HPDinf.parameters', nout=1).flatten()
                    hpd_sup = di.oc.eval('oo_.HPDsup.parameters', nout=1).flatten()
                    for i, name in enumerate(param_names_est):
                        if i < len(hpd_inf):
                            mcmc_results['param_intervals'][name] = (float(hpd_inf[i]), float(hpd_sup[i]))
            except:
                pass
            
            # Extract shock std means if available
            try:
                has_shock_means = di.oc.eval('isfield(oo_.posterior_mean, "shocks_std")', nout=1)
                if has_shock_means:
                    shock_means = di.oc.eval('oo_.posterior_mean.shocks_std', nout=1).flatten()
                    n_shocks = int(di.oc.eval('M_.exo_nbr', nout=1))
                    for i in range(n_shocks):
                        name = di.oc.eval(f'deblank(M_.exo_names{{{i+1}}})', nout=1)
                        name = str(name).strip()
                        if i < len(shock_means):
                            mcmc_results['shock_means'][name] = float(shock_means[i])
            except:
                pass
                
        else:
            print("MCMC results NOT available (mh_replic=0)")
            print("To get posterior mean and intervals, run with mh_replic=250000")
            
    except Exception as e:
        print(f"Error checking MCMC results: {e}")
    
    return mcmc_results

# Check for MCMC results
mcmc_results = check_mcmc_results(di)

MCMC results NOT available (mh_replic=0)
To get posterior mean and intervals, run with mh_replic=250000


## 4. Generate Table 1A - Structural Parameters

In [10]:
def create_table_1A(reference_data, estimated_params, mcmc_results=None):
    """
    Create Table 1A comparing estimated vs reference values.
    
    Args:
        reference_data: Dictionary with reference values from paper
        estimated_params: Dictionary with estimated parameter values from Dynare
        mcmc_results: Optional MCMC results with means and intervals
    
    Returns:
        DataFrame with Table 1A structure
    """
    rows = []
    
    for param_symbol, values in reference_data.items():
        dynare_name, prior_distr, prior_mean, prior_sd, post_mode, post_mean, post_5, post_95 = values
        
        # Get estimated mode from Dynare
        estimated_mode = estimated_params.get(dynare_name, np.nan)
        
        # Get MCMC results if available
        estimated_mean = np.nan
        estimated_5 = np.nan
        estimated_95 = np.nan
        
        if mcmc_results and mcmc_results['available']:
            estimated_mean = mcmc_results['param_means'].get(dynare_name, np.nan)
            if dynare_name in mcmc_results['param_intervals']:
                estimated_5, estimated_95 = mcmc_results['param_intervals'][dynare_name]
        
        # Calculate difference from paper's posterior mode
        if not np.isnan(estimated_mode):
            diff_pct = ((estimated_mode - post_mode) / post_mode) * 100
        else:
            diff_pct = np.nan
        
        rows.append({
            'Parameter': param_symbol,
            'Dynare Name': dynare_name,
            'Prior Distr.': prior_distr,
            'Prior Mean': prior_mean,
            'Prior SD': prior_sd,
            'Post. Mode (Paper)': post_mode,
            'Post. Mode (Replicated)': estimated_mode,
            'Diff (%)': diff_pct,
            'Post. Mean (Paper)': post_mean,
            'Post. Mean (Replicated)': estimated_mean,
            'Post. 5% (Paper)': post_5,
            'Post. 5% (Replicated)': estimated_5,
            'Post. 95% (Paper)': post_95,
            'Post. 95% (Replicated)': estimated_95,
        })
    
    return pd.DataFrame(rows)

# Generate Table 1A
table_1A = create_table_1A(TABLE_1A_REFERENCE, estimated_params, mcmc_results)
print("Table 1A generated")

Table 1A generated


In [11]:
# Display Table 1A - Prior Distribution
print("="*90)
print("TABLE 1A - PRIOR AND POSTERIOR DISTRIBUTION OF STRUCTURAL PARAMETERS")
print("Smets & Wouters (2007), American Economic Review, Table 1A (p.593)")
print("="*90)
print("\n** PRIOR DISTRIBUTION **\n")

prior_cols = ['Parameter', 'Prior Distr.', 'Prior Mean', 'Prior SD']
print(table_1A[prior_cols].to_string(index=False))

TABLE 1A - PRIOR AND POSTERIOR DISTRIBUTION OF STRUCTURAL PARAMETERS
Smets & Wouters (2007), American Economic Review, Table 1A (p.593)

** PRIOR DISTRIBUTION **

 Parameter Prior Distr.  Prior Mean  Prior SD
       phi       Normal        4.00      1.50
   sigma_c       Normal        1.50      0.37
         h         Beta        0.70      0.10
      xi_w         Beta        0.50      0.10
   sigma_l       Normal        2.00      0.75
      xi_p         Beta        0.50      0.10
    iota_w         Beta        0.50      0.15
    iota_p         Beta        0.50      0.15
       psi         Beta        0.50      0.15
       Phi       Normal        1.25      0.12
      r_pi       Normal        1.50      0.25
       rho         Beta        0.75      0.10
       r_y       Normal        0.12      0.05
 r_Delta_y       Normal        0.12      0.05
    pi_bar        Gamma        0.62      0.10
beta_const        Gamma        0.25      0.10
     l_bar       Normal        0.00      2.00
 gamma_ba

In [12]:
# Display Table 1A - Posterior Distribution Comparison
print("\n** POSTERIOR DISTRIBUTION (Mode Comparison) **\n")

posterior_cols = ['Parameter', 'Post. Mode (Paper)', 'Post. Mode (Replicated)', 'Diff (%)']
print(table_1A[posterior_cols].to_string(index=False, float_format=lambda x: f'{x:.2f}' if pd.notna(x) else 'N/A'))


** POSTERIOR DISTRIBUTION (Mode Comparison) **

 Parameter  Post. Mode (Paper)  Post. Mode (Replicated)  Diff (%)
       phi                5.48                     5.49      0.15
   sigma_c                1.39                     1.40      0.37
         h                0.71                     0.71      0.34
      xi_w                0.73                     0.74      1.03
   sigma_l                1.92                     1.92     -0.01
      xi_p                0.65                     0.66      0.96
    iota_w                0.59                     0.59      0.34
    iota_p                0.22                     0.23      3.80
       psi                0.54                     0.55      1.34
       Phi                1.61                     1.61      0.31
      r_pi                2.03                     2.03     -0.03
       rho                0.81                     0.82      0.66
       r_y                0.08                     0.08      5.86
 r_Delta_y                0

In [13]:
# Display Table 1A - Full Posterior Comparison (if MCMC available)
if mcmc_results and mcmc_results['available']:
    print("\n** FULL POSTERIOR DISTRIBUTION (Mean and Intervals) **\n")
    full_cols = ['Parameter', 'Post. Mean (Paper)', 'Post. Mean (Replicated)', 
                 'Post. 5% (Paper)', 'Post. 5% (Replicated)',
                 'Post. 95% (Paper)', 'Post. 95% (Replicated)']
    print(table_1A[full_cols].to_string(index=False, float_format=lambda x: f'{x:.2f}' if pd.notna(x) else 'N/A'))
else:
    print("\n** POSTERIOR DISTRIBUTION (Full - from Paper only) **")
    print("Note: Run with mh_replic=250000 to replicate mean and intervals\n")
    full_posterior_cols = ['Parameter', 'Post. Mode (Paper)', 'Post. Mean (Paper)', 'Post. 5% (Paper)', 'Post. 95% (Paper)']
    print(table_1A[full_posterior_cols].to_string(index=False))


** POSTERIOR DISTRIBUTION (Full - from Paper only) **
Note: Run with mh_replic=250000 to replicate mean and intervals

 Parameter  Post. Mode (Paper)  Post. Mean (Paper)  Post. 5% (Paper)  Post. 95% (Paper)
       phi                5.48                5.74              3.97               7.42
   sigma_c                1.39                1.38              1.16               1.59
         h                0.71                0.71              0.64               0.78
      xi_w                0.73                0.70              0.60               0.81
   sigma_l                1.92                1.83              0.91               2.78
      xi_p                0.65                0.66              0.56               0.74
    iota_w                0.59                0.58              0.38               0.78
    iota_p                0.22                0.24              0.10               0.38
       psi                0.54                0.54              0.36               0.72


## 5. Generate Table 1B - Shock Processes

In [14]:
def create_table_1B(reference_data, estimated_params, shock_std_estimates, mcmc_results=None):
    """
    Create Table 1B comparing estimated vs reference shock process values.
    
    Args:
        reference_data: Dictionary with reference values from paper
        estimated_params: Dictionary with estimated parameter values from Dynare
        shock_std_estimates: Dictionary with shock std dev estimates
        mcmc_results: Optional MCMC results with means and intervals
    
    Returns:
        DataFrame with Table 1B structure
    """
    rows = []
    
    for param_symbol, values in reference_data.items():
        dynare_name, prior_distr, prior_mean, prior_sd, post_mode, post_mean, post_5, post_95 = values
        
        # Determine if this is a shock std dev or a regular parameter
        is_shock_std = param_symbol.startswith('sigma_')
        
        if is_shock_std:
            # Get estimated shock std from shock_std_estimates
            estimated_mode = shock_std_estimates.get(dynare_name, np.nan)
            
            # Get MCMC mean if available
            estimated_mean = np.nan
            if mcmc_results and mcmc_results['available']:
                estimated_mean = mcmc_results['shock_means'].get(dynare_name, np.nan)
        else:
            # Regular parameter
            estimated_mode = estimated_params.get(dynare_name, np.nan)
            
            # Get MCMC results if available
            estimated_mean = np.nan
            if mcmc_results and mcmc_results['available']:
                estimated_mean = mcmc_results['param_means'].get(dynare_name, np.nan)
        
        # Calculate difference from paper's posterior mode
        if not np.isnan(estimated_mode):
            diff_pct = ((estimated_mode - post_mode) / post_mode) * 100
        else:
            diff_pct = np.nan
        
        rows.append({
            'Parameter': param_symbol,
            'Dynare Name': dynare_name,
            'Type': 'Shock Std' if is_shock_std else 'Parameter',
            'Prior Distr.': prior_distr,
            'Prior Mean': prior_mean,
            'Prior SD': prior_sd,
            'Post. Mode (Paper)': post_mode,
            'Post. Mode (Replicated)': estimated_mode,
            'Diff (%)': diff_pct,
            'Post. Mean (Paper)': post_mean,
            'Post. Mean (Replicated)': estimated_mean,
            'Post. 5% (Paper)': post_5,
            'Post. 95% (Paper)': post_95,
        })
    
    return pd.DataFrame(rows)

# Generate Table 1B
table_1B = create_table_1B(TABLE_1B_REFERENCE, estimated_params, shock_std_estimates, mcmc_results)
print("Table 1B generated")

Table 1B generated


In [15]:
# Display Table 1B - Prior Distribution
print("="*90)
print("TABLE 1B - PRIOR AND POSTERIOR DISTRIBUTION OF SHOCK PROCESSES")
print("Smets & Wouters (2007), American Economic Review, Table 1B (p.594)")
print("="*90)
print("\n** PRIOR DISTRIBUTION **\n")

prior_cols = ['Parameter', 'Prior Distr.', 'Prior Mean', 'Prior SD']
print(table_1B[prior_cols].to_string(index=False))

TABLE 1B - PRIOR AND POSTERIOR DISTRIBUTION OF SHOCK PROCESSES
Smets & Wouters (2007), American Economic Review, Table 1B (p.594)

** PRIOR DISTRIBUTION **

Parameter Prior Distr.  Prior Mean  Prior SD
  sigma_a     InvGamma         0.1       2.0
  sigma_b     InvGamma         0.1       2.0
  sigma_g     InvGamma         0.1       2.0
  sigma_I     InvGamma         0.1       2.0
  sigma_r     InvGamma         0.1       2.0
  sigma_p     InvGamma         0.1       2.0
  sigma_w     InvGamma         0.1       2.0
    rho_a         Beta         0.5       0.2
    rho_b         Beta         0.5       0.2
    rho_g         Beta         0.5       0.2
    rho_I         Beta         0.5       0.2
    rho_r         Beta         0.5       0.2
    rho_p         Beta         0.5       0.2
    rho_w         Beta         0.5       0.2
     mu_p         Beta         0.5       0.2
     mu_w         Beta         0.5       0.2
   rho_ga         Beta         0.5       0.2


In [16]:
# Display Table 1B - Posterior Distribution Comparison
print("\n** POSTERIOR DISTRIBUTION (Mode Comparison) **\n")

posterior_cols = ['Parameter', 'Post. Mode (Paper)', 'Post. Mode (Replicated)', 'Diff (%)']

# Split into sections for cleaner display
print("--- Shock Standard Deviations ---")
sigma_df = table_1B[table_1B['Type'] == 'Shock Std']
print(sigma_df[posterior_cols].to_string(index=False, float_format=lambda x: f'{x:.2f}' if pd.notna(x) else 'N/A'))

print("\n--- AR(1) Persistence ---")
rho_params = ['rho_a', 'rho_b', 'rho_g', 'rho_I', 'rho_r', 'rho_p', 'rho_w', 'rho_ga']
rho_df = table_1B[table_1B['Parameter'].isin(rho_params)]
print(rho_df[posterior_cols].to_string(index=False, float_format=lambda x: f'{x:.2f}' if pd.notna(x) else 'N/A'))

print("\n--- MA Coefficients ---")
ma_params = ['mu_p', 'mu_w']
ma_df = table_1B[table_1B['Parameter'].isin(ma_params)]
print(ma_df[posterior_cols].to_string(index=False, float_format=lambda x: f'{x:.2f}' if pd.notna(x) else 'N/A'))


** POSTERIOR DISTRIBUTION (Mode Comparison) **

--- Shock Standard Deviations ---
Parameter  Post. Mode (Paper)  Post. Mode (Replicated)  Diff (%)
  sigma_a                0.45                     0.45      0.40
  sigma_b                0.24                     0.24      1.03
  sigma_g                0.52                     0.52      0.00
  sigma_I                0.45                     0.45      0.02
  sigma_r                0.24                     0.24     -0.07
  sigma_p                0.14                     0.14      0.80
  sigma_w                0.24                     0.24      1.83

--- AR(1) Persistence ---
Parameter  Post. Mode (Paper)  Post. Mode (Replicated)  Diff (%)
    rho_a                0.95                     0.96      0.92
    rho_b                0.18                     0.18      1.36
    rho_g                0.97                     0.98      0.64
    rho_I                0.71                     0.71     -0.06
    rho_r                0.12                

In [17]:
# Display Table 1B - Full Posterior from Paper
print("\n** POSTERIOR DISTRIBUTION (Full - from Paper) **\n")

full_posterior_cols = ['Parameter', 'Post. Mode (Paper)', 'Post. Mean (Paper)', 'Post. 5% (Paper)', 'Post. 95% (Paper)']
print(table_1B[full_posterior_cols].to_string(index=False))


** POSTERIOR DISTRIBUTION (Full - from Paper) **

Parameter  Post. Mode (Paper)  Post. Mean (Paper)  Post. 5% (Paper)  Post. 95% (Paper)
  sigma_a                0.45                0.45              0.41               0.50
  sigma_b                0.24                0.23              0.19               0.27
  sigma_g                0.52                0.53              0.48               0.58
  sigma_I                0.45                0.45              0.37               0.53
  sigma_r                0.24                0.24              0.22               0.27
  sigma_p                0.14                0.14              0.11               0.16
  sigma_w                0.24                0.24              0.20               0.28
    rho_a                0.95                0.95              0.94               0.97
    rho_b                0.18                0.22              0.07               0.36
    rho_g                0.97                0.97              0.96            

## 6. Verification Summary

In [18]:
# Compute verification statistics
def compute_verification_stats(table_df, tolerance=5.0):
    """
    Compute verification statistics for a table.
    
    Args:
        table_df: DataFrame with 'Diff (%)' column
        tolerance: Acceptable percentage difference
    
    Returns:
        Dictionary with verification statistics
    """
    valid_diffs = table_df['Diff (%)'].dropna()
    
    n_total = len(table_df)
    n_estimated = len(valid_diffs)
    n_within_tol = (valid_diffs.abs() <= tolerance).sum()
    
    return {
        'total_params': n_total,
        'estimated_params': n_estimated,
        'within_tolerance': n_within_tol,
        'pass_rate': (n_within_tol / n_estimated * 100) if n_estimated > 0 else 0,
        'mean_abs_diff': valid_diffs.abs().mean() if len(valid_diffs) > 0 else np.nan,
        'max_abs_diff': valid_diffs.abs().max() if len(valid_diffs) > 0 else np.nan,
    }

# Verification for Table 1A
stats_1A = compute_verification_stats(table_1A)

# Verification for Table 1B
stats_1B = compute_verification_stats(table_1B)

print("="*70)
print("VERIFICATION SUMMARY")
print("="*70)
print(f"\nTolerance: 5% difference from paper's posterior mode")
print("\n--- Table 1A (Structural Parameters) ---")
print(f"  Total parameters: {stats_1A['total_params']}")
print(f"  Successfully estimated: {stats_1A['estimated_params']}")
print(f"  Within tolerance: {stats_1A['within_tolerance']}/{stats_1A['estimated_params']}")
print(f"  Pass rate: {stats_1A['pass_rate']:.1f}%")
print(f"  Mean absolute difference: {stats_1A['mean_abs_diff']:.2f}%")
print(f"  Max absolute difference: {stats_1A['max_abs_diff']:.2f}%")

print("\n--- Table 1B (Shock Processes) ---")
print(f"  Total parameters: {stats_1B['total_params']}")
print(f"  Successfully estimated: {stats_1B['estimated_params']}")
print(f"  Within tolerance: {stats_1B['within_tolerance']}/{stats_1B['estimated_params']}")
print(f"  Pass rate: {stats_1B['pass_rate']:.1f}%")
print(f"  Mean absolute difference: {stats_1B['mean_abs_diff']:.2f}%")
print(f"  Max absolute difference: {stats_1B['max_abs_diff']:.2f}%")
print("="*70)

VERIFICATION SUMMARY

Tolerance: 5% difference from paper's posterior mode

--- Table 1A (Structural Parameters) ---
  Total parameters: 19
  Successfully estimated: 19
  Within tolerance: 18/19
  Pass rate: 94.7%
  Mean absolute difference: 1.21%
  Max absolute difference: 5.86%

--- Table 1B (Shock Processes) ---
  Total parameters: 17
  Successfully estimated: 17
  Within tolerance: 16/17
  Pass rate: 94.1%
  Mean absolute difference: 0.97%
  Max absolute difference: 5.94%


In [19]:
# Highlight parameters with largest differences
print("\nParameters with largest differences (Table 1A):")
print("-" * 60)

table_1A_sorted = table_1A.dropna(subset=['Diff (%)']).sort_values('Diff (%)', key=abs, ascending=False)
for _, row in table_1A_sorted.head(5).iterrows():
    print(f"  {row['Parameter']:12s}  Paper: {row['Post. Mode (Paper)']:6.2f}  Replicated: {row['Post. Mode (Replicated)']:6.2f}  Diff: {row['Diff (%)']:+5.1f}%")

print("\nParameters with largest differences (Table 1B):")
print("-" * 60)

table_1B_sorted = table_1B.dropna(subset=['Diff (%)']).sort_values('Diff (%)', key=abs, ascending=False)
for _, row in table_1B_sorted.head(5).iterrows():
    print(f"  {row['Parameter']:12s}  Paper: {row['Post. Mode (Paper)']:6.2f}  Replicated: {row['Post. Mode (Replicated)']:6.2f}  Diff: {row['Diff (%)']:+5.1f}%")


Parameters with largest differences (Table 1A):
------------------------------------------------------------
  r_y           Paper:   0.08  Replicated:   0.08  Diff:  +5.9%
  iota_p        Paper:   0.22  Replicated:   0.23  Diff:  +3.8%
  l_bar         Paper:  -0.10  Replicated:  -0.10  Diff:  +3.1%
  alpha         Paper:   0.19  Replicated:   0.19  Diff:  +1.5%
  psi           Paper:   0.54  Replicated:   0.55  Diff:  +1.3%

Parameters with largest differences (Table 1B):
------------------------------------------------------------
  rho_r         Paper:   0.12  Replicated:   0.13  Diff:  +5.9%
  sigma_w       Paper:   0.24  Replicated:   0.24  Diff:  +1.8%
  rho_b         Paper:   0.18  Replicated:   0.18  Diff:  +1.4%
  rho_ga        Paper:   0.52  Replicated:   0.53  Diff:  +1.2%
  sigma_b       Paper:   0.24  Replicated:   0.24  Diff:  +1.0%


## 7. Export Results

In [ ]:
# Export tables to CSV
output_dir = Path.cwd()

# Table 1A
table_1A.to_csv(output_dir / 'table_1A_replication.csv', index=False)
print(f"Table 1A saved to: {output_dir / 'table_1A_replication.csv'}")

# Table 1B
table_1B.to_csv(output_dir / 'table_1B_replication.csv', index=False)
print(f"Table 1B saved to: {output_dir / 'table_1B_replication.csv'}")

In [ ]:
# Create formatted tables for paper-style display
print("\n" + "="*95)
print("FORMATTED TABLE 1A - Prior and Posterior Distribution of Structural Parameters")
print("="*95)
print("\n{:<12} {:>10} {:>8} {:>8}   {:>8} {:>8} {:>10} {:>10}".format(
    '', 'Prior', '', '', 'Posterior', '', '', ''))
print("{:<12} {:>10} {:>8} {:>8}   {:>8} {:>8} {:>10} {:>10}".format(
    'Parameter', 'Distr.', 'Mean', 'SD', 'Mode', 'Mean', '5%', '95%'))
print("-"*95)

for _, row in table_1A.iterrows():
    print("{:<12} {:>10} {:>8.2f} {:>8.2f}   {:>8.2f} {:>8.2f} {:>10.2f} {:>10.2f}".format(
        row['Parameter'],
        row['Prior Distr.'],
        row['Prior Mean'],
        row['Prior SD'],
        row['Post. Mode (Paper)'],
        row['Post. Mean (Paper)'],
        row['Post. 5% (Paper)'],
        row['Post. 95% (Paper)']
    ))

In [ ]:
print("\n" + "="*95)
print("FORMATTED TABLE 1B - Prior and Posterior Distribution of Shock Processes")
print("="*95)
print("\n{:<12} {:>10} {:>8} {:>8}   {:>8} {:>8} {:>10} {:>10}".format(
    '', 'Prior', '', '', 'Posterior', '', '', ''))
print("{:<12} {:>10} {:>8} {:>8}   {:>8} {:>8} {:>10} {:>10}".format(
    'Parameter', 'Distr.', 'Mean', 'SD', 'Mode', 'Mean', '5%', '95%'))
print("-"*95)

for _, row in table_1B.iterrows():
    print("{:<12} {:>10} {:>8.2f} {:>8.2f}   {:>8.2f} {:>8.2f} {:>10.2f} {:>10.2f}".format(
        row['Parameter'],
        row['Prior Distr.'],
        row['Prior Mean'],
        row['Prior SD'],
        row['Post. Mode (Paper)'],
        row['Post. Mean (Paper)'],
        row['Post. 5% (Paper)'],
        row['Post. 95% (Paper)']
    ))

## 8. Cleanup

In [ ]:
# Close Dynare/Octave session
di.close()
print("Octave session closed")
print("\nReplication complete!")

---

## Notes on Replication

### Current Configuration
- **mh_replic=0**: Only posterior mode is computed (fast)
- Shock standard deviations extracted from `M_.Sigma_e` diagonal

### To Get Full Posterior (Mean and Intervals)
Modify `usmodel.mod` line 207 to use:
```matlab
estimation(..., mh_replic=250000, ...);
```

This will run MCMC sampling and provide:
- Posterior mean
- 5% and 95% HPD intervals

**Warning**: MCMC with 250,000 draws takes significantly longer (~30-60 min).